# How to rediscover the Higgs boson yourself - with Machine Learning!
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roy-cruz/CROEM-ML2025/blob/master/exercises/higgs_classification.ipynb)

This notebook uses ATLAS Open Data http://opendata.atlas.cern to show you the steps to apply Machine Learning in search for the Higgs boson!

ATLAS Open Data provides open access to proton-proton collision data at the LHC for educational purposes. ATLAS Open Data resources are ideal for high-school, undergraduate and postgraduate students.

Notebooks are web applications that allow you to create and share documents that can contain for example:
1. live code
2. visualisations
3. narrative text

Notebooks are a perfect platform to develop Machine Learning for your work, since you'll need exactly those 3 things: code, visualisations and narrative text!

We're interested in Machine Learning because we can design an algorithm to figure out for itself how to do various analyses, potentially saving us countless human-hours of design and analysis work.

Machine Learning use within high energy physics includes:
* particle tracking
* particle identification
* signal/background classification
* and more!

This notebook will focus on signal/background classification.

By the end of this notebook you will be able to:
1. run machine learning models to classify signal and background
2. know some things you can change to improve your machine learning models

This analysis loosely follows the [discovery of the Higgs boson by ATLAS](https://www.sciencedirect.com/science/article/pii/S037026931200857X) (mostly Section 4 and 4.1)

## Running a Jupyter notebook

To run the whole Jupyter notebook, in the top menu click "Run" -> "Run all".

To propagate a change you've made to a piece of code, click "Run" -> "Run after".

You can also run a single code cell, by using the keyboard shortcut Shift+Enter.

---

## 🔬 Physics Motivation

In the Standard Model (SM), the **Higgs boson** is the quantum manifestation of the Higgs field, responsible for giving mass to other elementary particles via **electroweak symmetry breaking**. It was finally discovered in 2012 by the ATLAS and CMS experiments at the LHC.

One of the cleanest decay channels for identifying the Higgs boson is:

The Higgs boson decays as $H \rightarrow ZZ^* \rightarrow \ell^+ \ell^- \ell^+ \ell^-$.


- Here, the Higgs decays into two Z bosons, one of which may be off-shell (*virtual*, denoted by $Z^*$).
- Each Z then decays into a pair of leptons (electrons or muons),
- This results in **four final-state leptons**: a rare but striking signature.

> The branching ratio for $H \rightarrow ZZ^* \rightarrow 4\ell$ is small, but the final state is very clean and reconstructible.

---

## ⚠️ The Problem: Signal vs. Background

Our task is to identify these Higgs decays (**signal**) in the presence of similar-looking processes (**background**).

### ✅ Signal:
- **Process:** $gg \rightarrow H \rightarrow ZZ^* \rightarrow 4\ell$
- **Dataset label:** `"ggH125_ZZ4lep"`

This is the target process we want to identify. It represents **Higgs boson production via gluon-gluon fusion** ($gg$), followed by its decay into two Z bosons (one possibly off-shell), each of which decays into a pair of leptons. It produces a small peak at **125 GeV** in the 4-lepton invariant mass distribution.

### ❌ Background:
- **Process:** $q\bar{q} \rightarrow ZZ \rightarrow 4\ell$
- **Dataset label:** `"llll"`

This is a Standard Model process in which a **quark and antiquark pair** ($q\bar{q}$) annihilate to produce a pair of Z bosons, again decaying into four leptons. It mimics the same final state as the signal but does **not involve a Higgs boson**. These events form a broad continuum in the invariant mass spectrum and can overwhelm the signal.


### Feynman Diagrams

#### ✅ Signal Process: $gg \rightarrow H \rightarrow ZZ^* \rightarrow 4\ell$

In [ ]:
from IPython.display import Image, display, Markdown

display(Image("pictures/signal.png", width=500))


#### ❌ Background Process: $q\bar{q} \rightarrow ZZ \rightarrow 4\ell$


In [ ]:
display(Image("pictures/background.png", width=500))


## Get started!

We're going to be using a number of tools to help us:
* pandas: lets us store data as dataframes, a format widely used in Machine Learning
* numpy: provides numerical calculations such as histogramming
* matplotlib: common tool for making plots, figures, images, visualisations

Importing some basic libraries we need

In [ ]:
import pandas as pd  # to store data as dataframe
import numpy as np  # for numerical calculations such as histogramming
import matplotlib.pyplot as plt  # for plotting

Setting a seed for replicability.

In [ ]:
seed_value = 42
from numpy.random import seed
seed(seed_value)

We will use two (MC) datasets, one containing signal (i.e. `ggH125_ZZ4lep`) and one containing background (i.e. `llll`). We specify them and then load them into data frames.

In [ ]:
# In this notebook we only process the main signal ggH125_ZZ4lep and the main background llll,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["llll", "ggH125_ZZ4lep"]#, 'data'] # 'data' is not background it is... well... data
# signal: ZZ -> 4 leptons
# background: 4 leptons

Dataset:
https://www.kaggle.com/datasets/meirinevans/4lepton?select=ggH125_ZZ4lep.csv

In [ ]:
# get data from files

DataFrames = {}  # Define empty dictionary to hold DataFrames
for s in samples:  # Loop over sample names
    DataFrames[s] = pd.read_csv(f"data/{s}.csv")  # Load CSV from local 'data/' folder

We clean the data by getting rid of samples which do not obey lepton flavor conservation laws (i.e. only $Z \to ee$ and $Z \to \mu\mu$ are allowed). Note that, by convention, we arbitrarily assign $e$ a label of 11 and the $\mu$ a label of 12. Therefore only the decay of two Zs which result in an end product where the sum of labels equal to 44, 48 and 53 are allowed, given that they correspond to processes where 2 e's or 2 mu's are produced for each Z decay.

📌 TASK: Filter the dataset to retain only events that conserve lepton flavor, such as those with four electrons, four muons, or two electrons and two muons. Use the lepton type columns and apply a custom function to keep only events where the sum of lepton type codes equals 44, 48, or 52.

In [ ]:
# cut on lepton type
def cut_lep_type(lep_type_0, lep_type_1, lep_type_2, lep_type_3):
    # first lepton is [0], 2nd lepton is [1] etc
    # for an electron lep_type is 11
    # for a muon lep_type is 13
    # only want to keep events where one of eeee, mumumumu, eemumu
    sum_lep_type = lep_type_0 + lep_type_1 + lep_type_2 + lep_type_3
    if sum_lep_type == 44 or sum_lep_type == 48 or sum_lep_type == 53:
        return True
    else:
        return False


📌 TASK: Filter the dataset to retain only events that conserve lepton flavor, such as those with four electrons, four muons, or two electrons and two muons. Use the lepton type columns and apply a custom function to keep only events where the sum of lepton type codes equals 44, 48, or 52.

Charge must also be conserved. Z is neutral so the sum of the charge once Z decays must equal to 0.

**📌 TASK:**
Implement a charge conservation filter to keep only events
where the total lepton charge (sum of all four leptons) equals zero.
This reflects the fact that Z bosons are neutral and decay into oppositely charged lepton pairs.


In [ ]:
# Apply charge cut
for s in samples:
    DataFrames[s] = DataFrames[s][
        np.vectorize(cut_lep_charge)(
            DataFrames[s].lep_charge_0,
            DataFrames[s].lep_charge_1,
            DataFrames[s].lep_charge_2,
            DataFrames[s].lep_charge_3,
        )
    ]

Each event has 4 leptons and they are organized in descdending pt. This means lep 0 has the most pt, and lep 3 had the least.

In [ ]:
leppt0 = DataFrames["llll"].lep_pt_0
leppt1 = DataFrames["llll"].lep_pt_1
leppt2 = DataFrames["llll"].lep_pt_2
leppt3 = DataFrames["llll"].lep_pt_3

# Format the data for machine learning
As input values, for each event we will use the pt of the 2nd and 3rd leptons. Note that we could also take the pt of the other ones instead, or of all of them. There is nothing special about this selection. In general, it would be better to choose all of them as each datapoint would then contain more information for the event.

In [ ]:
# ML_inputs = ["lep_pt_0", "lep_pt_1", "lep_pt_2", "lep_pt_3"]
ML_inputs = ["lep_pt_1", "lep_pt_2"]

We now organize the data. First we extract all the features (lepton 1 and 2 pt)

 📌 TASK: Prepare the input features for the machine learning model
Loop through all Monte Carlo samples (excluding real data "data" sample), and for each one, extract the input variables defined in `ML_inputs`. Append these to a list, and then concatenate them into a single array X.


Now we extract all the corresponding event labels (1=event, 0=background)

In [ ]:
all_y = [] # Define empty list that will contain all features for the MC

for s in samples: # loop over the different samples
    if s != "data": # only MC should pass this if statement
        if "H125" in s: # only signal MC
            all_y.append(np.ones(DataFrames[s].shape[0]))
            # signal events labelled with 1
        else: # only background MC
            all_y.append(np.zeros(DataFrames[s].shape[0]))
            # background events labelled with 0

y = np.concatenate(all_y)
# concatenate the list of labels into a single 1d array of labels, called y

Now we split the data into the data we will use to train the model, and the data we will use to test it once it is trained. This is done automatically using the `train_test_split function` provided by `sklearn.model_selection`.

In [ ]:
# This will split your data into train-test sets: 67%-33%.
# It will also shuffle entries so you will not get the first 67% of X for training
# and the last 33% for testing.
# This is particularly important in cases where you load all signal events first
# and then the background events.

# Here we split our data into two independent samples.
# The split is to create a training and testing set.
# The first will be used for classifier training and the second to evaluate its performance.

from sklearn.model_selection import train_test_split

# make train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.33,
                                                    random_state=seed_value)

We normalize the data by using the `StandardScaler` function, which makes it so that our data is scaled to have a mean of 0 and a standard deviatinon of 1 before being fed into the machine learning model.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # initialise StandardScaler

# Fit only to the training data.
# Now scaler knows enough information about the testing dataset to do the normalization.
# However, it still hasn't done said normalization.
print(scaler.__dict__)
scaler.fit(X_train) # scaler will know how to transform the training data now
print(scaler.__dict__)

Applying transformations to the data and storing it in `X_train_scaled`

In [ ]:
X_train_scaled = scaler.transform(X_train) # Transform the data used the scaler which knows how to do this
print(X_train_scaled[:5])

Applying scaler transformation to X_test and X. Note that we do not FIT the testing dataset, only transform it based on the fit we did of the training dataset. We assume both datasets come from the same distribution, but because there is some level of noise, the "picture" they give of this distribution is slighly altered. We therefore settle on only normalizing based on the "picture" the training dataset gives us, as it will serve as the reference the model will base its learning on.

In [ ]:
X_test_scaled = scaler.transform(X_test) # You only TRANSFORM the testing set, not FIT
print(X_test_scaled[:5])

Comparing...

In [ ]:
print(X_train_scaled.mean())
print(X_test_scaled.mean())
print()
print(X_train_scaled.std())
print(X_test_scaled.std())

Standardization -> mean = 0, sigma = 1

Normalize -> Area under curve is 1

# Random Forest

📌 TASK: Train a Random Forest Classifier
Use the hyperparameters: max_depth, n_estimators, random_state=seed_value, criterion and choose the values for them based on the documentation https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.**html**

Double click for a description of the gini criterion...
<!-- > The `gini` criterion is a way of measuring the quality of a split in a decision tree. It is based on the Gini impurity, which is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. The Gini impurity can be computed by summing the probability of each item being chosen times the probability of a mistake in categorizing that item. It reaches its minimum (zero) when all cases in the node fall into a single target category.
> The gini criterion is one of the options for the criterion parameter in the RandomForestClassifier class of the sklearn.ensemble python package. The other options are entropy and log_loss, which are both based on the Shannon information gain. The default value for the criterion parameter is gini. -->

In [ ]:
# See how well the classifier does
print(accuracy_score(y_test, y_pred_RF))

# Neural Network

We will be using Pytorch for our NN

In [ ]:
import torch # import PyTorch
import torch.nn as nn # import PyTorch neural network
import torch.nn.functional as F # import PyTorch neural network functional
from torch.autograd import Variable # create variable from tensor
import torch.utils.data as Data # create data from tensors

Make variables for various PyTorch neural network hyper-parameters. Hyper-parameters refer to parameters which are not trainable and which we have control over outside of training.

In [ ]:
# Defining the hyperparameters
epochs = 25                   # number of training epochs or times we will go through the whole dataset whent training
batch_size = 32               # number of samples per batch
input_size = len(ML_inputs)   # Number of features
num_classes = 2               # Number of output classes. In this case signal or background
hidden_size = 5               # Number of nodes at the hidden layer
learning_rate = 0.001         # The speed of convergence
verbose = True                # flag for printing out stats at each epoch
torch.manual_seed(seed_value) # set random seed for PyTorch

The features that we will be using are contained withing `ML_inputs`

In [ ]:
print(ML_inputs)
# The information of each event which will be fed through the NN
# The NN will use this info to try to predict if the event corresponds
# to a background event or a signal event

Creating tensors, variables, datasets and loaders to build our neutral network in PyTorch. We keep some events for validation.

<!-- > Tensors are the central data structure in PyTorch. They are multi-dimensional arrays that can store elements of a single data type, such as floats, integers, booleans, etc. Tensors can run on CPUs or GPUs or other hardware accelerators to speed up computation.
>Tensors are similar to NumPy’s ndarrays, but they have some additional features and advantages:
> - Tensors can track gradients and support automatic differentiation, which is essential for building and training neural networks.
> - Tensors can leverage the torch.distributed package to support parallel and distributed computing across multiple machines and devices.
> - Tensors can interoperate with other Python libraries and frameworks, such as NumPy, SciPy, Pandas, etc.

> Variables are a deprecated concept in PyTorch. They used to be wrappers around tensors that supported automatic differentiation and gradient tracking. However, since PyTorch 0.4.0, tensors have been merged with variables, and there is no need to use the Variable class anymore.
> You can also use the torch.autograd package to perform more advanced operations on tensors and gradients, such as creating custom functions, disabling gradient tracking, computing higher-order derivatives, etc. -->

Notes:
- In older versions of PyTorch, the Variable class was used to track operations on tensors. However, in recent versions, Variable is no longer necessary, and tensors can be used directly.
- A TensorDataset is a PyTorch dataset that holds the input features and corresponding labels.
- A DataLoader is responsible for loading the data in batches for training. The batch_size parameter specifies how many samples per batch to load, and shuffle=True indicates that the data should be reshuffled at every epoch (iteration over the entire dataset).

📌 TASK: Convert the training data (X_train_scaled and y_train) into PyTorch tensors and wrap them as Variables. Then, split the first 1000 events as validation data, and the rest as training data for the neural network.


Tensors are similar to numpy arrays, just with more features that make them easy to use for ML.

We now organize our data into two sets:
- Training dataset: Data model will learn from
- Validation dataset: Data we will use to see how good our model is with data it hasn't seen before.

Once we make that division of the data, we put together the events in the training/validation dataset with their corresponding labels (signal or background), and put this pair into an object called a Tensor Dataset. You can think about this object as just another way to organize the same data we have.

NOTE:
In the tutorial, they convert the above defined tensors into something called a "Variable" first. Variables were used in older versions of Pytorch and were basically like tensors, but with extra features. However, these days, tensors already include the features that were once only a part of Variables, so it is not neccesary to first the tensor into a variable. Now we can go straight from a tensor to a dataset object.

In [ ]:
train_data = Data.TensorDataset(
    X_train_nn_var, y_train_nn_var
)  # create training dataset
valid_data = Data.TensorDataset(
    X_valid_var, y_valid_var
)  # create validation dataset

train_loader = Data.DataLoader(
    dataset=train_data,  # PyTorch Dataset
    batch_size=batch_size,  # how many samples per batch to load
    shuffle=True,
)  # data reshuffled at every epoch

valid_loader = Data.DataLoader(
    dataset=valid_data,  # PyTorch Dataset
    batch_size=batch_size,  # how many samples per batch to load
    shuffle=True,
)  # data reshuffled at every epoch


As its training, the model can't read the TensorDatasets directly. We need to use a `DataLoader`, which will take care of handling the data and feeding it into the NN.

(If you've used Keras before, this would be the equivalent of a data generators.)

📌 TASK: Using the already defined Variables, create PyTorch TensorDatasets for training and validation. Then, define DataLoaders for both datasets using the variable `batch_size`. Make sure to shuffle the data in both loaders.**bold text**

In [ ]:
print(type(train_data))
print(type(train_loader))

We now define the NN we will be using. It will be a simple, fully connected NN, with two hidden layers, both with the same number of neurons (`hidde_dim`). We use the ReLU activation function.

Notes:
- The softmax function is a mathematical function used to convert a vector of real numbers into a probability distribution. In PyTorch, the softmax function is provided as a built-in function torch.nn.functional.softmax() in the torch.nn.functional module.
- The softmax function ensures that the output values range between 0 and 1 and that the sum of the output values is equal to 1, making it suitable for representing a probability distribution.
- The `nn.Module` class contains useful functions for a NN. Instead of having to define all of them ourselves, we give this new class, `Classifier_MLP`, this class so that it inherits all of those useful functions and definitions. This is a fundamental feature of classes!
- The `super().__init__()` just executes the initialization function of the `nn.Module` class. Remember, some variables are defined by `nn.Module` and inherited by `Classifier_MLP`, so we still need to initialize them when creating a new instance of this class. In other words, this is done to ensure that the child class inherits the attributes and methods of the parent class.

The output of this model will be a `out_dim` dimensional vector in which the first element will represent the probability of the event being a signal, and the second element of it being background.

In [ ]:
class Classifier_MLP(nn.Module): # define Multi-Layer perceptron
    def __init__(self, in_dim, hidden_dim, out_dim): # initialise
        # Initialization method of the nn.Module class instance that
        # is inherited.
        super().__init__() # lets you avoid referring to the base class explicitly

        # Here we define the different layers that will be part of our model

        self.h1 = nn.Linear(in_dim, hidden_dim) # hidden layer 1\
        self.out = nn.Linear(hidden_dim, out_dim) # output layer
        self.out_dim = out_dim # output layer dimension

    def forward(self, x): # order of the layers
        # Neural network structure:
        # input layer -> h1 -> out
        x = F.relu(self.h1(x)) # relu activation function for hidden layer
        x = self.out(x) # no activation function for output layer

        return x, F.softmax(x, dim=1) # SoftMax function; dim=1 -> softmax applied along each row

We need to specify that we're using the `Classifier_MLP` model that we specified above and pass it the parameters it requires. We also specfy which optimizer we'll use to train our model. Here we use the Stochastic Gradient Descent (SGD) optimiser.

In [ ]:
# call Classifier_MLP class
NN_clf = Classifier_MLP(
    in_dim=input_size, hidden_dim=hidden_size, out_dim=num_classes
)
# optimize model parameters
optimizer = torch.optim.SGD(
    NN_clf.parameters(), lr=learning_rate
)

We now train the NN. This loop optimizes the parameters of the NN by going through the full training data set multiple times. Each loop is called an epoch. Note that during each epoch not all of the data is read at once. It is read in batches of pre-determined size defined by `train_loader`. Note that because `shuffle=True`, the full datasets will be shuffled on each epoch, so that we're not optimising over an identical sequence of samples in every loop. This reduces bias.

(As you read this code, keep in mind that we are using stochastic gradient descent to optimize the model. This means we need to compute the gradient (or slope) of the Loss function in parameter space.)

The loss function used is cross entropy. It is given by:
$$
    L = -\sum_{i}t_i \ln(p_i)
$$
where $t_i$ is the truth value for the $i$'th class and $p_i$ is the softmax probability for the $i$'th class. In this case:
$$
    L = -t_{\text{sig}} \ln(p_{\text{sig}}) - t_{\text{bg}} \ln(p_{\text{bg}})
$$
The update to the weights is given by:
$$
    W \to W - \eta\frac{\partial L}{\partial W}
$$
Where $W$ is the matrix of all the model's weights, $\eta$ is the learning rate, $L$ is the loss function and
$$\frac{\partial}{\partial W}= [\nabla_{\vec w^{(1)}}, \nabla_{\vec w^{(2)}}, \dots, \nabla_{\vec w^{(N)}}]^T$$

In [ ]:
# define empty list for epoch, train_loss, valid_loss, accuracy
_results = []
for epoch in range(epochs): # loop over the dataset multiple times

    # training loop for this epoch
    NN_clf.train() # set the model into training mode

    train_loss = 0.0 # start training loss counter at 0
    # loop over train_loader

    #-----------#-----------#-----------#-----------#-----------#-----------#-----------
    # Looping over batches

    for batch, (x_train_batch, y_train_batch) in enumerate(train_loader):
        '''
        batch -> batch number
        x_train_batch -> subset of training data in batch
        y_train_batch -> subset of training labels corresponding to x_train_batch in current batch
        '''
        NN_clf.zero_grad() # Set the gradients to zero before backpropagation because PyTorch accumulates the gradients
        out, prob = NN_clf(x_train_batch) # Get output and probability on this training batch
        loss = F.cross_entropy(out, y_train_batch) # calculate loss as cross entropy

        loss.backward() # compute dloss/dx
        optimizer.step() # updates the parameters

        train_loss += loss.item() * x_train_batch.size(0) # add to counter for training loss
        # loss.item -> average of loss over each data point
        # x_train_batch.size(0) -> size of dataset
        # Multiplying both gives us the sum of the loss of all the datapoints in this batch

    #-----------#-----------#-----------#-----------#-----------#-----------#-----------
    train_loss /= len(train_loader.dataset)# divide train loss by length of train_loader
    # train_loss is the average of the loss taken over the dataset

    if verbose: # if verbose flag set to True
        print("Epoch: {}, Train Loss: {:4f}".format(epoch, train_loss))

    #-----------#-----------#-----------#-----------#-----------#-----------#-----------
    # validation loop for this epoch
    NN_clf.eval() # set the model into evaluation mode
    with torch.no_grad(): # turn off the gradient calculations

        correct = 0
        valid_loss = 0 # start counters for number of correct and validation loss
        for i, (x_valid_batch, y_valid_batch) in enumerate(valid_loader): # loop over validation loader
            out, prob = NN_clf(x_valid_batch) # get output and probability on this validation batch
            loss = F.cross_entropy(out, y_valid_batch) # compute loss as cross entropy

            valid_loss += loss.item() * x_valid_batch.size(0) # add to counter for validation loss

            preds = prob.argmax(dim=1, keepdim=True) # get predictions
            correct += (
                preds.eq(y_valid_batch.view_as(preds)).sum().item()
            ) # count number of correct

        valid_loss /= len(valid_loader.dataset) # divide validation loss by length of validation dataset; i.e. get the average over the dataset
        accuracy = correct / len(valid_loader.dataset) # calculate accuracy as number of correct divided by total

        if verbose: # if verbose flag set to True
            print("Validation Loss: {:4f}, Validation Accuracy: {:4f}".format(valid_loss, accuracy))


        # create output row:
        _results.append([epoch, train_loss, valid_loss, accuracy])

results = np.array(_results) # make array of results
print("Finished Training")
print("Final validation error: ", 100.0 * (1-accuracy), "%")

In [ ]:
train_loss = np.array(_results)[:,1]
val_loss = np.array(_results)[:,2]

In [ ]:
plt.plot(train_loss, label="Training loss")
plt.plot(val_loss, label="Validation loss")
plt.legend()
plt.show()

In [ ]:
val_acc = np.array(_results)[:,3]

📌 TASK: Plot validation accuracy

Our model is now trained! We now proceed to show it data it has never seen to see how it performs.

In [ ]:
X_test_tensor = torch.as_tensor(X_test_scaled, dtype=torch.float)# Make tensor from X_test_scaled
y_test_tensor = torch.as_tensor(y_test, dtype=torch.long) # Make tensor from y_test

# Remember: We don't need to use variables
X_test_var, y_test_var = Variable(X_test_tensor), Variable(y_test_tensor) # make variables from tensors

out, prob = NN_clf(X_test_var) # Get output and probabilities from X_test
y_pred_NN = prob.cpu().detach().numpy().argmax(axis=1) # get signal/background predictions

In [ ]:
print(y_pred_NN[0:50])
print(y_test_tensor[0:50])

Now we see how well this NN classifier did using accuracy_score.

📌 TASK: Calculate yourself using sklearn.metrics lib

# Overfitting Check

Comparing a machine learning model’s output distribution for the training and testing set is a popular way in High Energy Physics to check for overfitting. The compare_train_test() method will plot the shape of the machine learning model’s decision function for each class, as well as overlaying it with the decision function in the training set.

If overfitting were present, the dots (test set) would be very far from the bars (training set).

In [ ]:
# This script shows you how to create python functions and save
# them in an output file to use in a kernel. To see how to use them in
# a kernel go here: https://www.kaggle.com/rtatman/import-functions-from-kaggle-script/

def compare_train_test(clf, X_train, y_train, X_test, y_test, xlabel):
    decisions = [] # list to hold decisions of classifier
    for X,y in ((X_train, y_train), (X_test, y_test)): # train and test
        if hasattr(clf, "predict_proba"): # if predict_proba function exists
            d1 = clf.predict_proba(X[y<0.5])[:, 1] # background
            d2 = clf.predict_proba(X[y>0.5])[:, 1] # signal
        else: # predict_proba function doesn't exist
            X_tensor = torch.as_tensor(X, dtype=torch.float) # make tensor from X_test_scaled
            y_tensor = torch.as_tensor(y, dtype=torch.long) # make tensor from y_test
            X_var, y_var = Variable(X_tensor), Variable(y_tensor) # make variables from tensors
            d1 = clf(X_var[y_var<0.5])[1][:, 1].cpu().detach().numpy() # background
            d2 = clf(X_var[y_var>0.5])[1][:, 1].cpu().detach().numpy() # signal
        decisions += [d1, d2] # add to list of classifier decision

    highest_decision = max(np.max(d) for d in decisions) # get maximum score
    bin_edges = [] # list to hold bin edges
    bin_edge = -0.1 # start counter for bin_edges
    while bin_edge < highest_decision: # up to highest score
        bin_edge += 0.1 # increment
        bin_edges.append(bin_edge)

    plt.hist(decisions[0], # background in train set
             bins=bin_edges, # lower and upper range of the bins
             density=True, # area under the histogram will sum to 1
             histtype='stepfilled', # lineplot that's filled
             color='blue', label='Background (train)', # Background (train)
            alpha=0.5 ) # half transparency
    plt.hist(decisions[1], # background in train set
             bins=bin_edges, # lower and upper range of the bins
             density=True, # area under the histogram will sum to 1
             histtype='stepfilled', # lineplot that's filled
             color='orange', label='Signal (train)', # Signal (train)
            alpha=0.5 ) # half transparency

    hist_background, bin_edges = np.histogram(decisions[2], # background test
                                              bins=bin_edges, # number of bins in function definition
                                              density=True ) # area under the histogram will sum to 1

    scale = len(decisions[2]) / sum(hist_background) # between raw and normalised
    err_background = np.sqrt(hist_background * scale) / scale # error on test background

    width = 0.1 # histogram bin width
    center = (bin_edges[:-1] + bin_edges[1:]) / 2 # bin centres

    plt.errorbar(x=center, y=hist_background, yerr=err_background, fmt='o', # circles
                 c='blue', label='Background (test)' ) # Background (test)

    hist_signal, bin_edges = np.histogram(decisions[3], # siganl test
                                          bins=bin_edges, # number of bins in function definition
                                          density=True ) # area under the histogram will sum to 1
    scale = len(decisions[3]) / sum(hist_signal) # between raw and normalised
    err_signal = np.sqrt(hist_signal * scale) / scale # error on test background

    plt.errorbar(x=center, y=hist_signal, yerr=err_signal, fmt='o', # circles
                 c='orange', label='Signal (test)' ) # Signal (test)

    plt.xlabel(xlabel) # write x-axis label
    plt.ylabel("Arbitrary units") # write y-axis label
    plt.legend() # add legend


In [ ]:
#Overfitting check for random forest
compare_train_test(
    RF_clf, X_train_scaled, y_train, X_test_scaled, y_test, "Random Forest output"
)

📌 TASK: Plot overfitting check for NN


In [ ]:
# Ovefitting check for NN


In [ ]:
compare_train_test?

# Model Comparison
Precision -> ratio of all true positives to all things that were classified positive
$$\text{precision} = \frac{\text{TP}}{\text{TP} + \text{FP}} = \frac{\text{# of events correctly classified as signal}}{\text{Total # of events classified as signal}} $$
Recall -> ratio of all true positives to all things that should've been classified as positive
$$\text{recall} = \frac{\text{TP}}{\text{TP} + \text{FN}} = \frac{\text{# of events correctly classified as signal}}{\text{Total # of events which were actual signals}}$$

📌 TASK: Calculate precision, recall, F1-score with sklearn.metrics

In [ ]:
decisions_rf = RF_clf.predict_proba(X_test_scaled)[:,1] # Get the decisions of the random forest

In [ ]:
print(decisions_rf[:30]) # each probability of each event corresponding to signal as predicted by random forest

In [ ]:
decisions_nn = NN_clf(X_test_var)[1][:,1].cpu().detach().numpy() # get the decisions of the neural network

In [ ]:
print(decisions_nn[:30]) # each probability of each event corresponding to signal as predicted by NN

# ROC Curve

In [ ]:
# histogram of background events/negative event probabilities as predicted by the RF
plt.hist(
    decisions_rf[y_test == 0],
    histtype="step",
    bins=50,
    label="Background Events"
)  # plot background
# histogram of signal events/positive event probabilities as predicted by the RF
plt.hist(
    decisions_rf[y_test == 1],
    histtype="step",
    bins=50,
    linestyle="dashed",
    label="Signal Events",
)  # plot signal
plt.xlabel("Threshold")  # x-axis label
plt.ylabel("Number of Events")  # y-axis label
plt.semilogy()  # make the y-axis semi-log
plt.legend()  # draw the legend

ROC curve -> plot of the recall (or TP) vs. FP which we can as well alter the threshold (i.e. line in the above graph for which events on the right correspond to signal and events to the left corresponds to background.

In [ ]:
from sklearn.metrics import roc_curve
# get false positive rates (FPRs), true positive rates (TPRs) and threshold for random forest
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, decisions_rf)
# get false positive rates (FPRs), true positive rates (TPRs) and threshold for NN
fpr_nn, tpr_nn, thresholds_nn = roc_curve(y_test, decisions_nn)

📌 TASK: Plot ROC curve for NN and Random Forest

### Defining the Approximate Median Significance (AMS) Function

In high-energy physics, we are often interested not just in how accurately we classify events, but in how statistically significant a detected signal is over the expected background. The Approximate Median Significance (AMS) provides a measure of this significance using the predicted rates of signal (true positives) and background (false positives) events.

The AMS function approximates the median significance with which a signal can be distinguished from background, and is particularly useful when working with rare signals where traditional metrics like accuracy or precision may be misleading.

The formula used here is:

$$
\text{AMS} = \sqrt{2\left[(\text{TPR}+\text{FPR} + b_r)\ln\left(1 + \frac{\text{TPR}}{\text{FPR} + b_r}\right)-\text{TPR}\right]}
$$

Where:
- **TPR** is the sum of the signal (true positive) weights.
- **FPR** is the sum of the background (false positive) weights.
- \( b_r \) is a small regularization term (typically \( 10^{-6} \)) to prevent division by zero or taking the log of zero.

This function returns a value in units of Gaussian standard deviations (σ), indicating how significant the signal is above the background. In the context of discoveries at the LHC, a 5σ AMS value corresponds to the commonly used threshold for a statistically significant discovery.


📌 TASK: Define AMS function and plot it for NN and Random Forest

In [ ]:
ams_rf = AMS(tpr_rf, fpr_rf)
ams_nn = AMS(tpr_nn, fpr_nn)

In [ ]:
plt.plot(thresholds_rf, ams_rf, label="Random Forest")  # plot random forest AMS
plt.plot(
    thresholds_nn, ams_nn, linestyle="dashed", label="Neural Network"
)  # plot neural network AMS
plt.xlabel("Threshold")  # x-axis label
plt.ylabel("AMS")  # y-axis label
plt.title("AMS with $b_r=0.001$")  # add plot title
plt.legend()  # add legend

The threshold we should use should maximize the AMS.

# Applying To Experimental Data, Additional material

## Challenge: Apply models to experimental data

In [ ]:
# Reading data file
data_DF = pd.read_csv("data/data.csv")
data_DF

In [ ]:
# Applying cuts
data_DF = data_DF[np.vectorize(cut_lep_type)(
        data_DF.lep_type_0,
        data_DF.lep_type_1,
        data_DF.lep_type_2,
        data_DF.lep_type_3,
    )
]
data_DF

In [ ]:
data_DF = data_DF[np.vectorize(cut_lep_charge)(
        data_DF.lep_charge_0,
        data_DF.lep_charge_1,
        data_DF.lep_charge_2,
        data_DF.lep_charge_3,
    )
]
data_DF

In [ ]:
# Extracting only relevant data which will be used as the model input and
# converting to numpy array
X_data = data_DF[ML_inputs].to_numpy()

In [ ]:
# Scaling data
# Initializing Standard Scaler
scaler_data = StandardScaler()
scaler.fit(X_data)
print(scaler_data.__dict__)

In [ ]:
X_data_scaled = scaler.transform(X_data)

In [ ]:
# Random Forest predictions
y_data_RF = RF_clf.predict(X_data_scaled)
print(y_data_RF[0:30])

In [ ]:
# Formatting data to use in NN and putting it through NN
X_data_tensor = torch.as_tensor(X_data_scaled, dtype=torch.float)
y_data_NN = NN_clf(X_data_tensor)
print(y_data_NN[1][0:10])

In [ ]:
labels = ["background", "signal"]  # labels for simulated data
thresholds = []  # define list to hold random forest classifier probability predictions for each sample

for s in samples:  # loop over samples
    thresholds.append(RF_clf.predict_proba(scaler.transform(DataFrames[s][ML_inputs]))[:, 1])  # predict probabilities for each sample

plt.hist(thresholds, bins=np.arange(0, 0.8, 0.1), density=True, stacked=True, label=labels)  # plot simulated data

data_hist = np.histogram(RF_clf.predict_proba(X_data_scaled)[:, 1], bins=np.arange(0, 0.8, 0.1), density=True)[0]  # histogram the experimental data

scale = sum(RF_clf.predict_proba(X_data_scaled)[:, 1]) / sum(data_hist)  # get scale imposed by density=True

data_err = np.sqrt(data_hist * scale) / scale  # get error on experimental data

plt.errorbar(x=np.arange(0.05, 0.75, 0.1), y=data_hist, yerr=data_err, label="Data")  # plot the experimental data errorbars
plt.xlabel("Threshold")
plt.legend()

In [ ]:
# Number of signal/background events the random forest classifier is predicting
print(np.count_nonzero(y_data_RF == 1)) # signal
print((np.count_nonzero(y_data_RF == 0))) # background